In [12]:
import os
import numpy as np
import tensorflow as tf
import ncnn
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
classifier = tf.keras.models.load_model(os.path.join(
    "..",
    "miniCDDD_convert",
    "classifier.h5",
))

classifier.summary()

Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512)]             0         
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 128)               65664     
                                                                 
 classification_output (Dens  (None, 7)                903       
 e)                                                              
                                                                 
Total params: 329,223
Trainable params: 329,223
Non-trainable params: 0
_________________________________________________________________


In [14]:
cddd_emb = np.load("cddd_emb.npy")
cddd_emb.shape

(512,)

In [15]:
miniCDDD_out = classifier(cddd_emb.reshape((1, -1)))

In [16]:
def miniCDDD_infer(
    miniCDDD_in: ncnn.Mat,
) -> ncnn.Mat:
    with ncnn.Net() as net:
        net.opt.use_fp16_packed = False
        net.opt.use_fp16_storage = False
        net.opt.use_fp16_arithmetic = False

        net.load_param(os.path.join("model", "classifier.param"))
        net.load_model(os.path.join("model", "classifier.bin"))

        with net.create_extractor() as ex:
            ex.input("input_2_blob", miniCDDD_in)

            _, miniCDDD_out = ex.extract("classification_output_blob")

    return miniCDDD_out

In [24]:
ncnn_out = np.array(miniCDDD_infer(ncnn.Mat(cddd_emb))).reshape((1, -1))

In [25]:
cosine_similarity(miniCDDD_out, np.array(ncnn_out))

array([[0.99999994]], dtype=float32)

In [26]:
ncnn_out

array([[ 0.26657274, -0.02520834,  0.02151309, -0.02601999, -0.12428422,
         0.42039272,  0.04190039]], dtype=float32)